In [1]:
import pandas as pd
import os

# --- Path Settings ---
DIR_PREFIX = "./data/SF/"
CENTERS_LABELED_FILENAME = "centers_100_labeled.csv"
WAITING_POSITIONS_FILENAME = "waiting_positions.csv"
ERROR_RATE_STATISTICS_FILENAME = "error_rates.csv"

# --- Test Settings ---
GROUP_NUM = 3
CLIENTS_NUM_MAX_LIMIT = 15
DROP_DUPLICATE_REPORTS = False
assert GROUP_NUM * 2 < CLIENTS_NUM_MAX_LIMIT

In [2]:
centers = pd.read_csv(os.path.join(DIR_PREFIX, CENTERS_LABELED_FILENAME), index_col="cluster-id")
centers = centers["traffic-light"]
centers.describe()

count    100.00
mean       0.55
std        0.50
min        0.00
25%        0.00
50%        1.00
75%        1.00
max        1.00
Name: traffic-light, dtype: float64

In [3]:
waiting_positions = pd.read_csv(os.path.join(DIR_PREFIX, WAITING_POSITIONS_FILENAME))
waiting_positions.head()

,time,latitude,longitude,id,cluster-id
0,1211020534,37.77764,-122.43995,abmuyawm,0
1,1211020595,37.77763,-122.43996,abmuyawm,0
2,1211021066,37.77922,-122.43444,abmuyawm,1
3,1211022336,37.77364,-122.43808,abmuyawm,0
4,1211022569,37.77393,-122.43041,abmuyawm,0


In [4]:
waiting_positions["traffic-light"] = -1
for cluster_id in centers.index.to_list():
    waiting_positions.loc[waiting_positions["cluster-id"] == cluster_id, "traffic-light"] = centers[cluster_id]

waiting_positions = waiting_positions[waiting_positions["traffic-light"] >= 0]
waiting_positions.describe()

,time,latitude,longitude,cluster-id,traffic-light
count,7.169000e+03,7169.000000,7169.000000,7169.000000,7169.000000
mean,1.212012e+09,37.761167,-122.435798,282.382062,0.617938
std,5.884922e+05,0.021653,0.027070,286.447924,0.485925
min,1.211020e+09,37.701400,-122.499970,3.000000,0.000000
25%,1.211512e+09,37.748500,-122.455920,63.000000,0.000000
50%,1.211999e+09,37.766060,-122.430110,172.000000,1.000000
75%,1.212516e+09,37.779180,-122.414770,402.000000,1.000000
max,1.213086e+09,37.797410,-122.400000,1409.000000,1.000000


In [5]:
waiting_positions = waiting_positions[["id", "traffic-light"]]
if DROP_DUPLICATE_REPORTS:
    waiting_positions.drop_duplicates(inplace=True)

In [6]:
true_detections_n = waiting_positions.groupby("id")["traffic-light"].sum()
total_detections_n = waiting_positions.groupby("id")["traffic-light"].count()
total_detections_n.name = "total-times"
statistics = pd.concat([true_detections_n, total_detections_n], axis=1)
statistics["error_rate"] = 1.0 - (statistics["traffic-light"] / statistics["total-times"])
statistics.describe()

,traffic-light,total-times,error-rate
count,99.000000,99.000000,99.000000
mean,44.747475,72.414141,0.356746
std,36.691306,48.556723,0.191069
min,2.000000,4.000000,0.000000
25%,26.000000,43.500000,0.209914
50%,40.000000,64.000000,0.326087
75%,55.000000,88.500000,0.467967
max,325.000000,333.000000,0.913978


In [7]:
statistics.to_csv(os.path.join(DIR_PREFIX, "error_rates.csv"), columns=["error-rate"])